# MATH189: Homework 3

By Leah Seseri, Imangali Amangeldi

## Problem 1

### Problem 1, Part 1: Hypothesis testing

In [31]:
water <- read.table("Water.txt", fileEncoding = "UTF-16LE", header = TRUE)
t.test(water$bottom, water$surface, paired=TRUE)


	Paired t-test

data:  water$bottom and water$surface
t = 4.8638, df = 9, p-value = 0.0008911
alternative hypothesis: true mean difference is not equal to 0
95 percent confidence interval:
 0.043006 0.117794
sample estimates:
mean difference 
         0.0804 


### Problem 1, Part 2: Assumption Disscusion

The assumption of homoskedasticity is that for a two-sample test, the data from all groups have a common variance. So, if this assumption was violated and the the two groups have unequal variances then using this two-sample t-test that assumes equal variances would lead to wrong conclusions and could cause Type one errors where the null hypothesis is rejected when it is actually true. Also, Type two errors could increase where real differences could go undetected and incorrect confidence intervals could occur. Thus, if this assumption was violated a standard t-test is not reliable to produce accurate results.

# Problem 2

Load the required packages as well as dataset

In [32]:
# 1. Install the package
install.packages("HSAUR3", repos = "https://cloud.r-project.org")

# 2. Load the package and data
library(HSAUR3)
data("pottery")

# 3. Write out to CSV
write.csv(pottery,
          file = "pottery.csv",
          row.names = FALSE)

Warning message:
"package 'HSAUR3' is in use and will not be installed"


### Problem 2, Part 1: Calculating F-statistic

In [33]:
# 1. Subset to kilns 1,2,4,5
pot <- subset(pottery, kiln %in% c(1,2,4,5))

# 2. Identify the nine chemical columns
chemicals <- names(pot)[1:9]

# 3. Function to extract the F‐value from an aov object
getF <- function(var) {
  f <- aov(as.formula(paste(var, "~ factor(kiln)")), data=pot)
  summary(f)[[1]]["F value"][1]
}

# 4. Compute all nine F‐statistics at once
F_stats <- sapply(chemicals, getF)

# 5. View results
print(F_stats)

$`Al2O3.F value`
[1] 26.00087       NA

$`Fe2O3.F value`
[1] 154.3197       NA

$`MgO.F value`
[1] 97.7673      NA

$`CaO.F value`
[1] 53.50213       NA

$`Na2O.F value`
[1] 10.46517       NA

$`K2O.F value`
[1] 81.76186       NA

$`TiO2.F value`
[1] 14.65845       NA

$`MnO.F value`
[1] 52.75629       NA

$`BaO.F value`
[1] 0.4590205        NA



### Problem 2, Part 2: Calculating Bonferroni adjusted p-values

In [34]:
# Compute p‐values for each chemical
p_vals <- sapply(chemicals, function(var) {
  fit <- aov(as.formula(paste(var, "~ factor(kiln)")), data = pot)
  summary(fit)[[1]][["Pr(>F)"]][1]
})

# View p‐values
print(p_vals)

       Al2O3        Fe2O3          MgO          CaO         Na2O          K2O 
2.083495e-09 1.126525e-21 3.434369e-18 6.880285e-14 3.480171e-05 7.126204e-17 
        TiO2          MnO          BaO 
1.524685e-06 8.561655e-14 7.124818e-01 


In [35]:
# Adjust p‐values using Bonferroni correction
p_bonf <- p.adjust(p_vals, method = "bonferroni")

# View adjusted p‐values
print(p_bonf)

       Al2O3        Fe2O3          MgO          CaO         Na2O          K2O 
1.875146e-08 1.013872e-20 3.090932e-17 6.192257e-13 3.132154e-04 6.413583e-16 
        TiO2          MnO          BaO 
1.372217e-05 7.705489e-13 1.000000e+00 


Using a Bonferroni-corrected significance level of  
$$
\alpha_{\mathrm{BF}} = \frac{0.05}{9} \approx 0.0056,
$$  
we reject \(H_0\) for Al₂O₃, Fe₂O₃, MgO, CaO, Na₂O, K₂O, TiO₂, and MnO (all adjusted \(p<0.0056\)), indicating their mean concentrations differ across kilns 1, 2, 4 & 5.  
BaO, with a Bonferroni-adjusted \(p = 1.00\), fails to reject \(H_0\) and therefore shows no significant site-to-site variation.


In [36]:
# Set the significance level for Bonferroni correction
alpha_bf <- 0.05 / length(p_vals)

# Combine results into a data frame, for better readability,
# and adding rejection decision
results <- data.frame(
  Chemical   = chemicals,
  p_value    = p_vals,
  Bonferroni = p_bonf,
  Reject     = p_bonf < alpha_bf
)

# View the results
print(results)

      Chemical      p_value   Bonferroni Reject
Al2O3    Al2O3 2.083495e-09 1.875146e-08   TRUE
Fe2O3    Fe2O3 1.126525e-21 1.013872e-20   TRUE
MgO        MgO 3.434369e-18 3.090932e-17   TRUE
CaO        CaO 6.880285e-14 6.192257e-13   TRUE
Na2O      Na2O 3.480171e-05 3.132154e-04   TRUE
K2O        K2O 7.126204e-17 6.413583e-16   TRUE
TiO2      TiO2 1.524685e-06 1.372217e-05   TRUE
MnO        MnO 8.561655e-14 7.705489e-13   TRUE
BaO        BaO 7.124818e-01 1.000000e+00  FALSE


### Problem 2, Part 2: Calculating Benjamini-Hochberg adjusted p-values

In [37]:
# Adjust p‐values using Benjamini-Hochberg correction
p_bh <- p.adjust(p_vals, method = "BH")

# View adjusted p‐values
print(p_bh)

       Al2O3        Fe2O3          MgO          CaO         Na2O          K2O 
3.125243e-09 1.013872e-20 1.545466e-17 1.541098e-13 3.915192e-05 2.137861e-16 
        TiO2          MnO          BaO 
1.960309e-06 1.541098e-13 7.124818e-01 


Using a Benjamini–Hochberg procedure to control the false discovery rate at  
$$
q = 0.05,
$$  
we reject \(H_0\) for Al₂O₃, Fe₂O₃, MgO, CaO, Na₂O, K₂O, TiO₂, and MnO (all BH-adjusted \(p<0.05\)), indicating their mean concentrations differ across kilns 1, 2, 4 & 5.  
BaO, with a BH-adjusted \(p = 0.7125\), fails to reject \(H_0\) and therefore shows no significant site-to-site variation.


In [38]:
# Set the significance level for Benjamini-Hochberg correction
alpha <- 0.05

# Combine results into a data frame, for better readability
results <- data.frame(
  Chemical   = chemicals,
  p_value    = p_vals,
  BH         = p_bh,
  Reject     = p_bh < alpha
)

# View the results
print(results)

      Chemical      p_value           BH Reject
Al2O3    Al2O3 2.083495e-09 3.125243e-09   TRUE
Fe2O3    Fe2O3 1.126525e-21 1.013872e-20   TRUE
MgO        MgO 3.434369e-18 1.545466e-17   TRUE
CaO        CaO 6.880285e-14 1.541098e-13   TRUE
Na2O      Na2O 3.480171e-05 3.915192e-05   TRUE
K2O        K2O 7.126204e-17 2.137861e-16   TRUE
TiO2      TiO2 1.524685e-06 1.960309e-06   TRUE
MnO        MnO 8.561655e-14 1.541098e-13   TRUE
BaO        BaO 7.124818e-01 7.124818e-01  FALSE
